In [64]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

import re
import  os
import random
import time
import datetime
from dateutil.relativedelta import relativedelta

import numpy as np

In [2]:
AccountList=[['13302111561', 'Xnxdw199882']]

In [3]:
df = pd.read_csv('douban.csv')

In [4]:
df['year'] = df['date'].map(lambda x: x[:4])
df['month'] = df['date'].map(lambda x: '12' if len(x)<7 else x[5:7])
df['day'] = df['date'].map(lambda x: '15' if len(x)<10 else x[8:10])

In [5]:
df['index'] = df.apply(lambda x: '('+re.sub("[\!\%\[\]\，\。\·\：\、\!\t]", "", x['name'])+','+x['year']+','+x['month']+','+x['day']+')', axis=1)  

In [6]:
df['year'] = df['year'].map(lambda x: int(x))
df = df[df['year']>=2011]
df['year'] = df['year'].map(lambda x: str(x))

In [7]:
df.sample(10)

,Unnamed: 0,name,rating,id,sentiment,director,scriptwriter,starring,genre,date,total_people,running_time,year,month,day,index
1069,1258,昆虫总动员,8.1,23048775,1.000000e+00,"['海琳·吉罗', '托马斯·绍博']","['托马斯·绍博', '海琳·吉罗']",[],动画,2014-08-22(中国大陆),191902,89,2014,08,22,"(昆虫总动员,2014,08,22)"
1527,1731,北极大冒险,6.6,24752662,1.000000e+00,"['罗杰·斯波蒂斯伍德', '白兰度·奎利西']","['巴特·加维根', '休·赫德森']","['达科塔·高尤', '高兰·维斯耶克', '布丽姬·穆娜', '杜安·莫瑞', '彼得·麦...",剧情,2016-12-30(中国大陆),26896,92,2016,12,30,"(北极大冒险,2016,12,30)"
1158,1348,大喜临门,5.3,26280561,1.000000e+00,['黄朝亮'],"['李惠娟', '周元姣', '黄朝亮']","['猪哥亮', '林心如', '李东学', '寇世勋', '林美秀']",喜剧,2015-03-06(中国大陆),91923,104,2015,03,06,"(大喜临门,2015,03,06)"
879,1056,速度与激情6,7.7,6537500,6.183787e-11,['林诣彬'],"['克里斯·摩根', '盖瑞·斯科特·汤普森']","['范·迪塞尔', '保罗·沃克', '道恩·强森', '乔丹娜·布鲁斯特', '米歇尔·罗...",动作,2013-07-26(中国大陆),13683,130,2013,07,26,"(速度与激情6,2013,07,26)"
988,1174,早安，冬日海,4.8,25744252,9.999841e-01,['胡涵清'],"['秦铭禧', '甘世佳']","['张子文', '任容萱', '钱泳辰']",爱情,2014-01-10(中国大陆),2586,93,2014,01,10,"(早安冬日海,2014,01,10)"
1620,1825,碟仙前传,3.4,26950073,1.326239e-03,['高育新'],['高育新'],"['王右硕', '伊娜', '郭若寒', '王瑄', '明俊臣']",惊悚,2017-05-12(中国大陆),559677,90,2017,05,12,"(碟仙前传,2017,05,12)"
1045,1234,死亡邮件,3.5,25798808,1.000000e+00,['李彧'],"['于波', '周雅莉']","['任泉', '乔任梁', '张杨果而', '李彧', '周泓']",悬疑,2014-06-20(中国大陆),110,90,2014,06,20,"(死亡邮件,2014,06,20)"
1489,1690,神奇动物在哪里,7.8,25726614,1.000000e+00,['大卫·叶茨'],['J·K·罗琳'],"['埃迪·雷德梅恩', '凯瑟琳·沃特斯顿', '丹·福勒', '艾莉森·萨多尔', '科林...",剧情,2016-11-25(中国大陆),111,133,2016,11,25,"(神奇动物在哪里,2016,11,25)"
998,1185,江南爱情故事,5.1,25761527,9.999994e-01,['任鹏'],['卢冬红'],"['沈秋雨', '沈涛', '王欢', '呼延晓辉', '张恩齐']",爱情,2014-02-14(中国大陆),284,94,2014,02,14,"(江南爱情故事,2014,02,14)"
760,929,孤岛惊魂2,2.8,20380318,9.991008e-01,['钟继昌'],['方思行'],"['邓家佳', '陈志朋']",悬疑,2013-01-26(中国大陆),102,93,2013,01,26,"(孤岛惊魂2,2013,01,26)"


In [8]:
indexes = df['index'].tolist()
weibo_search = []
for i in indexes:
    tmp = i[1:-1].split(',')
    weibo_search.append(tuple(tmp))

In [9]:
weibo_search

[('年年有今日', '2011', '08', '19'),
 ('龙飞凤舞', '2012', '01', '13'),
 ('阴阳路六之凶周刊', '2017', '10', '27'),
 ('红岩 第2部', '2016', '05', '06'),
 ('蜘蛛侠', '2017', '09', '08'),
 ('黑白', '2016', '03', '18'),
 ('诺玛的十七岁', '2017', '09', '15'),
 ('鸿门宴', '2011', '11', '29'),
 ('爱了再爱', '2012', '05', '20'),
 ('跆拳道', '2016', '08', '18'),
 ('卧虎', '2017', '09', '30'),
 ('哆啦A梦:大雄的恐龙', '2017', '05', '30'),
 ('怪女孩', '2015', '02', '21'),
 ('梦之队', '2013', '09', '20'),
 ('钢铁侠', '2013', '05', '01'),
 ('樱桃', '2012', '04', '24'),
 ('追星家族', '2014', '03', '07'),
 ('鸟巢', '2015', '12', '18'),
 ('月亮', '2011', '06', '法国'),
 ('破冰', '2013', '04', '02'),
 ('圣经里的秘密', '2013', '12', '25'),
 ('过界', '2014', '02', '14'),
 ('诡计', '2016', '07', '13'),
 ('牛铃响叮当', '2017', '11', '01'),
 ('光之美少女充满秘密的玩具王国', '2013', '10', '26'),
 ('快枪手', '2016', '07', '15'),
 ('情定敖木仑', '2013', '03', '21'),
 ('大国手之扬州论枰', '2011', '08', '20'),
 ('迷城', '2015', '07', '30'),
 ('纳尼亚传奇黎明踏浪号', '2011', '01', '07'),
 ('创战纪', '2011', '01', '10'),
 ('喜羊羊与灰太狼之兔年顶呱呱', '2011',

In [10]:
def init_spider():
        url = "https://weibo.cn/search/mblog"
        browser = webdriver.Chrome('/home/eloise/anaconda3/pkgs/chromedriver_linux64/chromedriver')
        browser.get(url)
        time.sleep(10)
        list=random.choice(AccountList)
        browser.find_element_by_id("loginPassword").send_keys(list[1])
        browser.find_element_by_id("loginName").send_keys(list[0])
        browser.find_element_by_id("loginAction").click()
        time.sleep(10)
        return browser

In [11]:
def CalculateDate(year,month):
    fdate = datetime.date(int(year), int(month), 15)
    adate = fdate - relativedelta(months=6)
    
    fyear = str(year)
    fmonth = str(month)
    ayear = int(adate.year)
    amonth = int(adate.month)
    
    if ayear == 2010:
        ayear = 2011
        amonth = 1
        
    ayear = str(ayear)
    amonth = str(amonth)
    
    if len(fmonth)<2:
        fmonth = '0' + fmonth
    if len(amonth)<2:
        amonth = '0' + amonth
    
    return fyear,fmonth,ayear,amonth

In [28]:
def get_weibo(name, fyear, fmonth, ayear, amonth, day1, day2):
        browser.find_elements_by_xpath("/html/body/div[6]/form/div/input[2]")[0].send_keys(name)
        
        browser.find_elements_by_xpath("/html/body/div[6]/form/div/input[11]")[0].send_keys(ayear+amonth+day1)
        browser.find_elements_by_xpath("/html/body/div[6]/form/div/input[12]")[0].send_keys(fyear+fmonth+day2)
        browser.find_elements_by_xpath("/html/body/div[6]/form/div/input[14]")[0].click()
        browser.find_elements_by_xpath("/html/body/div[6]/form/div/input[15]")[0].click()
        
        try:
            number = int(browser.find_elements_by_css_selector("body > div:nth-child(5) > span")[0].text[1:-1])
        except:
            try:
                number = int(browser.find_elements_by_css_selector("body > div:nth-child(4) > span")[0].text[1:-1])
            except:
                return False
        
        total = 0
        comments = browser.find_elements_by_xpath('//a[contains(., "评论[")]')
        for comment in comments:
            try:
                total += int(comment.text[3:][:-1])
            except:
                total += int(comment.text[5:][:-1])
        comments = total
            
        total = 0
        forwards = browser.find_elements_by_xpath('//a[contains(., "转发[")]')
        for forward in forwards:
            total += int(forward.text[3:][:-1])
        forwards = total
        
        total = 0
        likes = browser.find_elements_by_xpath('//a[contains(., "赞[")]')
        for like in likes:
            total += int(like.text[2:][:-1])
        likes = total
        
        return number, comments, forwards, likes

In [32]:
def exec_spider(name, year, month, day):
        global browser

        fyear,fmonth,ayear,amonth=CalculateDate(year,month)
        browser.maximize_window()
        # 点击网页上的开始日期
        
        halfy = get_weibo(name, fyear, fmonth, ayear, amonth, '15', day)
        if halfy is False:
            return False
        
        browser.back()
        browser.refresh()
        
        halfago = get_weibo(name, ayear, amonth, ayear, amonth, '15', '15')
        if halfago is False:
            return False
        browser.back()
        browser.refresh()
        
        release = get_weibo(name, fyear, fmonth, fyear, fmonth, day, day)
        if release is False:
            return False
        
        return {
            'average': halfy,
            'difference': [release[0]-halfago[0], release[1]-halfago[1], release[2]-halfago[2], release[3]-halfago[3]]
        }

In [37]:
global browser
browser = init_spider()

In [33]:
result = []

In [62]:
flag = False
for item in weibo_search:
    if item[0] == '床下有人2':
        flag = True
        
    if flag is False:
        continue 
        
    status=exec_spider(item[0], item[1], item[2], item[3])
    
    if status is False:
        result.append({'average': [0,0,0,0], 'difference': [0,0,0,0]})
    else:
        result.append(status)
        
    try:
        browser.back()
        browser.refresh()
    except:
        time.sleep(100)
        browser = init_spider()
    
    print(item[0])
    print(status)
            
    time.sleep(10)

床下有人2
{'average': (130, 9086, 11061, 11982), 'difference': [-92, -382, -1107, -431]}
昆虫总动员
{'average': (341, 1421, 12918, 7913), 'difference': [-157, 0, 0, 0]}
下一站再爱你
{'average': (129, 1619, 3629, 4764), 'difference': [-69, -803, -2380, -1281]}
大丑
{'average': (313, 10373, 2982, 32443), 'difference': [-27, 0, 0, 0]}
美人邦
{'average': (270, 504, 552, 2213), 'difference': [-148, -128, 209, -1001]}
忘了去懂你
{'average': (271, 750, 4033, 4837), 'difference': [-194, -235, -3134, 80]}
猩球崛起黎明之战
{'average': (800, 1828, 3697, 2729), 'difference': [-528, -763, -1077, -472]}
青春.com
{'average': (53, 267, 298, 953), 'difference': [-11, -8, -19, 74]}
动物也疯狂
{'average': (351, 710, 459, 3698), 'difference': [-137, 0, 0, 0]}
大峰祖师
{'average': (295, 1064, 3778, 2182), 'difference': [-212, -827, -2718, -1536]}
摩登森林之美食总动员
{'average': (11, 201, 239, 54), 'difference': [-5, -44, -70, -18]}
分歧者异类觉醒
{'average': (341, 1346, 4908, 2973), 'difference': [-178, 0, 0, 0]}
美女与野兽
{'average': (919, 733, 699, 6603), 'difference

点对点
{'average': (896, 365, 423, 1798), 'difference': [-2, 0, 0, 0]}
十万伙急
{'average': (63, 1197, 2248, 647), 'difference': [-8, 87, 41, -354]}
疯狂外星人
{'average': (882, 149375, 1002438, 461257), 'difference': [0, 0, 0, 0]}
阳光总在风雨后
{'average': (905, 9206, 7290, 25364), 'difference': [0, 0, 0, 0]}
赤道
{'average': (897, 7483, 3490, 18559), 'difference': [0, 0, 0, 0]}
爱我就陪我看电影
{'average': (391, 7920, 16190, 58659), 'difference': [-339, -1736, -9092, -52257]}
踏雪寻梅
{'average': (866, 4098, 8640, 27876), 'difference': [1, 0, 0, 0]}
超能查派
{'average': (460, 3386, 4622, 10156), 'difference': [-227, -1091, -2173, -1790]}
分手不分手
{'average': (148, 2223, 341, 2098), 'difference': [-8, 0, 0, 0]}
夺命旅行
{'average': (64, 751, 1376, 5214), 'difference': [-30, -443, -928, -3707]}
旋风九日
{'average': (132, 236, 1456, 2040), 'difference': [-78, -42, -567, 214]}
岸边之旅
False
哺乳期的女人
{'average': (44, 498, 1321, 1810), 'difference': [-11, 17, 57, 392]}
阿里巴巴大盗奇兵
{'average': (55, 380, 1249, 66), 'difference': [-49, -344, -120

我的青春期
{'average': (352, 824, 680, 2699), 'difference': [-88, 1689, 2277, 7373]}
男二本色
{'average': (12, 397, 412, 612), 'difference': [-1, -63, -80, 133]}
我的少女时代
{'average': (884, 7081, 29928, 27733), 'difference': [-1, 0, 0, 0]}
饥饿游戏3嘲笑鸟（下）
{'average': (371, 2550, 7034, 8891), 'difference': [-272, -1432, -1327, -5157]}
玩命速递重启之战
{'average': (73, 2627, 4671, 1754), 'difference': [-41, 449, 311, -52]}
十月初五的月光
{'average': (805, 24577, 14589, 443155), 'difference': [-556, -16603, -9701, -359016]}
遭遇海明威
{'average': (80, 5143, 7164, 6337), 'difference': [-63, -4228, -6001, -5292]}
青春激荡的岁月
{'average': (33, 51, 127, 155), 'difference': [0, 0, 0, 0]}
坏蛋必须死
{'average': (812, 44044, 220771, 321273), 'difference': [-371, -41395, -207061, -272956]}
诡影迷情
False
火云端
{'average': (32, 514, 705, 767), 'difference': [2, 12, -343, 418]}
怦然星动
{'average': (859, 116101, 139377, 1415598), 'difference': [-35, 0, 0, 0]}
不可思异
{'average': (378, 35066, 28450, 345181), 'difference': [-237, -5111, -3918, -152241]}
杜拉拉追

人在驴途
{'average': (26, 95, 432, 432), 'difference': [0, 0, 0, 0]}
路边野餐
{'average': (864, 1313, 3273, 5821), 'difference': [-3, 0, 0, 0]}
惊天大逆转
{'average': (887, 583, 584, 3455), 'difference': [-1, 0, 0, 0]}
超能太阳鸭
{'average': (126, 2311, 1592, 8267), 'difference': [-95, 2284, 7075, -6371]}
假装看不见之电影大师
{'average': (17, 191, 198, 180), 'difference': [-11, -74, -139, -128]}
绝地逃亡
{'average': (813, 35359, 251539, 83114), 'difference': [-16, -24244, -199763, -40415]}
魔都凶音
{'average': (146, 1907, 3832, 5509), 'difference': [-105, -1701, -2542, -4765]}
巴霍巴利王（上）开端
{'average': (19, 3768, 7101, 2887), 'difference': [-12, -3003, -5678, 53]}
恐怖爱情故事之死亡公路
{'average': (19, 363, 426, 681), 'difference': [-14, -200, -250, -448]}
少年师爷之大禹宝藏
{'average': (22, 685, 947, 212), 'difference': [-20, -639, -913, -202]}
红色警戒999
{'average': (60, 3592, 18271, 810), 'difference': [-33, 450, -11332, -22]}
封神传奇
{'average': (681, 66859, 100237, 98544), 'difference': [-337, -37208, -39891, -32298]}
毒中毒
{'average': (511, 167

少年
{'average': (2518268, 49889, 161523, 168348), 'difference': [176129, 0, 19, 4]}
五女闹京城
{'average': (16, 232, 6682, 298), 'difference': [-13, -228, -3334, -291]}
生门
{'average': (632, 18003, 17329, 61019), 'difference': [-51, 0, 0, 0]}
功夫四侠
{'average': (15, 306, 348, 407), 'difference': [-7, -160, -167, -131]}
摆渡人
{'average': (903, 1846, 2594, 10548), 'difference': [-2, 0, 0, 0]}
铁道飞虎
{'average': (815, 7104, 8425, 12167), 'difference': [0, 0, 0, 0]}
有迹可循
{'average': (872, 2122, 17854, 19985), 'difference': [1, 0, 0, 0]}
失心者
{'average': (41, 2766, 2824, 10598), 'difference': [-23, 453, 664, 134]}
绝对控制
{'average': (183, 1469, 2736, 4806), 'difference': [-66, 0, 0, 0]}
冒牌卧底
{'average': (167, 6284, 9721, 24937), 'difference': [-109, -5111, -9384, -22707]}
爸爸的3次婚礼
{'average': (127, 3384, 4914, 12493), 'difference': [-95, -1961, -2798, -7006]}
你好疯子！
{'average': (689, 9757, 23046, 38666), 'difference': [-300, 419, -4408, 1696]}
那年夏天你去了哪里
{'average': (697, 14572, 1021569, 102901), 'difference'

摔跤吧！爸爸
{'average': (886, 18062, 28037, 194813), 'difference': [0, 0, 0, 0]}
咸鱼传奇
{'average': (74, 5576, 31003, 27536), 'difference': [-50, -1089, -5452, -1550]}
麻烦家族
{'average': (559, 57608, 598486, 1080782), 'difference': [-384, -51748, -581621, -1041793]}
毒诫
{'average': (115, 5024, 6053, 76108), 'difference': [-91, -4038, -5721, -49258]}
碟仙前传
{'average': (5, 551, 450, 1280), 'difference': [-1, -170, -16, -703]}
提着心吊着胆
{'average': (175, 5109, 2772, 14706), 'difference': [-109, 0, 0, 0]}
亚瑟王斗兽争霸
{'average': (3, 6, 10, 49), 'difference': [0, 0, 0, 0]}
超凡战队
{'average': (653, 2588, 2335, 21737), 'difference': [-489, 558, 329, 1825]}
20:16
{'average': (222, 73979, 1010517, 67276), 'difference': [-91, -8124, -9923, -2855]}
当怪物来敲门
{'average': (147, 2366, 3718, 9188), 'difference': [-63, -1381, -2124, -4258]}
不期而遇
{'average': (938, 1006763, 1011076, 659884), 'difference': [1, -999990, -999979, -503637]}
异星觉醒
{'average': (420, 10607, 16646, 33552), 'difference': [-233, 388, -4039, -4028]}
守护者世

In [65]:
a_number = []
a_comment = []
a_forward = []
a_like = []
b_number = []
b_comment = []
b_forward = []
b_like = []

for x in result:
    if x is False:
        a_number.append(np.nan)
        a_comment.append(np.nan)
        a_forward.append(np.nan)
        a_like.append(np.nan)
        b_number.append(np.nan)
        b_comment.append(np.nan)
        b_forward.append(np.nan)
        b_like.append(np.nan)
    else:
        a_number.append(x['average'][0])
        a_comment.append(x['average'][1])
        a_forward.append(x['average'][2])
        a_like.append(x['average'][3])
        b_number.append(x['difference'][0])
        b_comment.append(x['difference'][1])
        b_forward.append(x['difference'][2])
        b_like.append(x['difference'][3])

In [68]:
df.columns

Index(['Unnamed: 0', 'name', 'rating', 'id', 'sentiment', 'director',
       'scriptwriter', 'starring', 'genre', 'date', 'total_people',
       'running_time', 'year', 'month', 'day', 'index'],
      dtype='object')

In [82]:
df2 = df
df2['a_number'] = pd.Series(a_number)
df2['a_comment'] = pd.Series(a_comment)
df2['a_forward'] = pd.Series(a_forward)
df2['a_like'] = pd.Series(a_like)
df2['b_number'] = pd.Series(b_number)
df2['b_comment'] = pd.Series(b_comment)
df2['b_forward'] = pd.Series(b_forward)
df2['b_like'] = pd.Series(b_like)
df2.sample(20)

,Unnamed: 0,name,rating,id,sentiment,director,scriptwriter,starring,genre,date,...,day,index,a_number,a_comment,a_forward,a_like,b_number,b_comment,b_forward,b_like
1699,1908,我的爸爸是森林之王,6.8,27046450,1.000000,"['本·斯塔森', '杰里米·德格鲁森']","['鲍勃·巴伦', '卡尔·布伦克尔']","['帕皮·福克纳', '克里斯托弗·L·帕森', '玛莉芙·赫林顿', '泰伦斯·斯通', ...",喜剧,2017-10-14(中国大陆),...,14,"(我的爸爸是森林之王,2017,10,14)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,1150,赤焰战场2,7.0,7916027,1.000000,['迪恩·帕里索特'],"['乔·霍贝尔', '埃里希·霍贝尔', '库里·哈米尔', '沃伦·埃利斯']","['布鲁斯·威利斯', '玛丽·露易斯·帕克', '凯瑟琳·泽塔-琼斯', '海伦·米伦',...",喜剧,2013-11-15(中国大陆),...,15,"(赤焰战场2,2013,11,15)",109.0,53188.0,134530.0,315531.0,-56.0,-8302.0,-4716.0,-10849.0
1267,1461,炸掉银行经理,6.3,26323407,0.999983,['丹尼·德拉托雷'],['阿尔贝托·马里尼'],"['路易斯·托萨尔', '戈雅·托莱多', '哈维尔·古铁雷斯', '埃尔薇拉·明戈斯', ...",惊悚,2015-09-25(西班牙),...,25,"(炸掉银行经理,2015,09,25)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,773,惊魂游戏,3.7,6536267,0.999525,['周耀武'],['周耀武'],"['胡兵', '莫小棋', '李玲玉', '王双宝', '朴努植']",悬疑,2012-01-24(中国大陆),...,24,"(惊魂游戏,2012,01,24)",470243.0,1773161.0,377181.0,2264903.0,-8.0,0.0,1.0,7.0
1205,1397,对风说爱你,6.5,25830802,1.000000,['王童'],"['郭筝', '李佳颖', '林智祥', '吴境']","['杨祐宁', '郭采洁', '郭碧婷', '胡宇威', '柯佳嬿']",剧情,2015-06-26(中国大陆),...,26,"(对风说爱你,2015,06,26)",658.0,3249.0,8139.0,20238.0,-361.0,0.0,0.0,0.0
1371,1570,魔宫魅影,4.7,26220725,1.000000,['叶伟民'],"['文隽', '杨梅媛', '李晶凌']","['林心如', '杨祐宁', '任达华', '黄幻', '景岗山']",剧情,2016-04-29(中国大陆),...,29,"(魔宫魅影,2016,04,29)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1547,1751,西游伏妖篇,5.6,25801066,1.000000,['徐克'],"['周星驰', '李思臻', '徐克']","['吴亦凡', '林更新', '姚晨', '林允', '包贝尔']",喜剧,2017-01-28(中国大陆),...,28,"(西游伏妖篇,2017,01,28)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1563,1767,完美有多美,5.4,26668750,1.000000,['梁栋'],['梁栋'],"['姜武', '蒋勤勤', '李乃文', '朱旭', '傅颖']",喜剧,2017-02-17(中国大陆),...,17,"(完美有多美,2017,02,17)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048,1237,美国骗局,6.6,6873657,1.000000,['大卫·O·拉塞尔'],"['埃里克·辛格尔', '大卫·O·拉塞尔']","['克里斯蒂安·贝尔', '布莱德利·库珀', '艾米·亚当斯', '杰瑞米·雷纳', '詹...",剧情,2014-07-04(中国大陆),...,04,"(美国骗局,2014,07,04)",911.0,1780.0,1034.0,4919.0,-3.0,0.0,0.0,0.0
710,877,毕业那年,3.8,10759842,0.000401,['姚宇'],['仧縯'],"['顾莉雅', '朱文超', '叶青青', '王仲欣', '丁汀']",剧情,2012-09-21(中国大陆),...,21,"(毕业那年,2012,09,21)",37.0,601.0,957.0,2901.0,-25.0,-299.0,-343.0,-1845.0


In [83]:
df3 = df2.dropna(how='any')
df3.count()

Unnamed: 0      734
name            734
rating          734
id              734
sentiment       734
director        734
scriptwriter    734
starring        734
genre           734
date            734
total_people    734
running_time    734
year            734
month           734
day             734
index           734
a_number        734
a_comment       734
a_forward       734
a_like          734
b_number        734
b_comment       734
b_forward       734
b_like          734
dtype: int64

In [85]:
df3.columns

Index(['Unnamed: 0', 'name', 'rating', 'id', 'sentiment', 'director',
       'scriptwriter', 'starring', 'genre', 'date', 'total_people',
       'running_time', 'year', 'month', 'day', 'index', 'a_number',
       'a_comment', 'a_forward', 'a_like', 'b_number', 'b_comment',
       'b_forward', 'b_like'],
      dtype='object')

In [86]:
df3.drop(['Unnamed: 0', 'date', 'index'],axis=1,inplace=True)
df3.sample(10)

/home/eloise/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,name,rating,id,sentiment,director,scriptwriter,starring,genre,total_people,running_time,...,month,day,a_number,a_comment,a_forward,a_like,b_number,b_comment,b_forward,b_like
776,特警判官,6.3,3387483,0.999974,['皮特·特拉维斯'],"['亚历克斯·嘉兰', '约翰·瓦格纳', '卡洛斯·埃斯克拉']","['卡尔·厄本', '奥莉薇·瑟尔比', '琳娜·海蒂', '道比·欧帕瑞', '兰利·柯克...",动作,2400,90,...,03,01,587.0,2889.0,2761.0,26279.0,-223.0,0.0,0.0,0.0
581,北角,6.3,6514087,0.003866,['朱少宇'],['秋艳'],"['钟淑慧', '吴岱融', '林隽健', '曾幸']",剧情,8364,90,...,11,04,885.0,616.0,990.0,2752.0,-1.0,0.0,0.0,0.0
558,挪威的森林,5.9,3168086,1.000000,['陈英雄'],"['陈英雄', '村上春树']","['松山研一', '菊地凛子', '水原希子', '高良健吾', '玉山铁二']",剧情,3528,133,...,09,16,228.0,1761.0,4910.0,7798.0,-97.0,-462.0,-3545.0,-2519.0
863,赤警威龙,5.8,3256172,1.000000,['沃尔特·希尔'],['亚历山卓·卡蒙'],"['西尔维斯特·史泰龙', '姜成镐', '莎拉·夏希', '阿德沃尔·阿吉纽依-艾格拜吉'...",动作,49490,91,...,07,09,808.0,5449.0,2306.0,9986.0,-245.0,344.0,235.0,1996.0
825,被解救的姜戈,8.5,6307447,1.000000,['昆汀·塔伦蒂诺'],['昆汀·塔伦蒂诺'],"['杰米·福克斯', '莱昂纳多·迪卡普里奥', '克里斯托弗·沃尔兹', '塞缪尔·杰克逊...",剧情,10862,163,...,05,12,240684.0,3679.0,3798.0,22419.0,0.0,-184.0,-126.0,-1672.0
762,洛克王国2圣龙的心愿,5.7,20443571,1.000000,['于胜军'],['曹大伟'],"['林颖', '孙闻']",动画,60796,89,...,01,31,879.0,10814.0,13447.0,47623.0,-1.0,29.0,17.0,211.0
600,深度谜案,6.7,1950192,1.000000,['阿莱克斯·德·拉·伊格莱西亚'],"['阿莱克斯·德·拉·伊格莱西亚', '豪尔赫·格力加耶切瓦利亚']","['伊利亚·伍德', '约翰·赫特', '蕾欧诺·瓦特林', '朱莉·考科斯', '吉姆·卡特']",悬疑,137981,108,...,12,31,295.0,1064.0,3778.0,2182.0,-212.0,-827.0,-2718.0,-1536.0
705,超凡蜘蛛侠,7.2,6082518,1.000000,['马克·韦布'],"['艾里克斯·库兹曼', '罗伯托·奥奇', '杰夫·皮克纳', '詹姆斯·范德比尔特', ...","['安德鲁·加菲尔德', '艾玛·斯通', '杰米·福克斯', '戴恩·德哈恩', '坎贝尔...",动作,821,142,...,05,04,132.0,236.0,1456.0,2040.0,-78.0,-42.0,-567.0,214.0
778,青春雷锋,2.5,11627079,0.999979,['刘一君'],"['谭仲池', '彭海燕']","['胡家华', '孙健淇', '张江诗琴', '种丹妮', '温亚南']",剧情,42308,101,...,03,04,902.0,2081.0,2831.0,6851.0,0.0,0.0,0.0,0.0
857,萨米大冒险2,6.6,10540210,1.000000,['本·斯塔森'],['多米尼克·帕里斯'],"['帕特·卡洛尔', '卡洛斯·麦卡勒斯二世', '辛达·亚当斯', '迪诺·安德拉德', ...",动画,2848,92,...,06,28,816.0,6127.0,1572.0,156533.0,-5.0,15516.0,25054.0,-69472.0


In [87]:
df3.to_csv('weibo.csv')